# PA3b Sentiment Classification
#### Applied Machine Learning
Grpup 39: Sebastian Kölbel & Min Ze Teh

We begin by importing libraries needed for the task

In [134]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import LinearSVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# for converting training and test datasets into matrices
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB

We create a function for preprocessing the csv-files. Thes preprocessing steps include:
* Making text all lower case
* Removing leading and closing white-space
* Make sure punctuations are separated from words

In [128]:
def read_sentiments(doc_file):
    with open(doc_file, 'r', encoding='utf-8') as f:
        new_lines = []
        for line in f:
            line = line.lower().removesuffix('\n').strip().replace('.',' . ')
            new_lines.append(line.split('\t'))
                
        return pd.DataFrame(new_lines,columns=['sentiment','text'])

We then separate the input values from the output values. The crowdsourced data needed some more preprocessing. There we too many values for the sentiments all consisting of typos. We checked how many errors there were, however there were only 75 errors out of more than 10 000 rows so the errors were dropped to save time.

In [129]:
df = read_sentiments('Data/crowdsourced_train.csv').drop(0)
cs_training = df[df['sentiment'].isin(['positive','negative','neutral'])]
print('Dropped rows in crowdsourced data:',len(df)-len(cs_training))
X_cs_train = cs_training.drop('sentiment', axis=1)
Y_cs_train = cs_training['sentiment']

gold_training = read_sentiments('Data/gold_train.csv').drop(0)
X_gold_train = gold_training.drop('sentiment', axis=1)
Y_gold_train = gold_training['sentiment']


testing = read_sentiments('Data/test.csv').drop(0)
X_test = testing.drop('sentiment', axis=1)
Y_test = testing['sentiment']

Dropped rows in crowdsourced data: 75


In [137]:
def train_document_classifier(X, Y):
    pipeline = make_pipeline( TfidfVectorizer(), MultinomialNB() )
    pipeline.fit(X, Y)
    return pipeline

In [148]:
vec = TfidfVectorizer()
myvec = vec.fit_transform(X_cs_train)
mnb = MultinomialNB()

Y_cs_train
#mnb.fit(myvec,Y_cs_train)
#train_document_classifier(X_cs_train,Y_cs_train)

1        positive
2        positive
3        negative
4         neutral
5        positive
           ...   
10672    positive
10673    positive
10674     neutral
10675    negative
10676    positive
Name: sentiment, Length: 10601, dtype: object